###**Alunos**
- Vandelson Elias (vemf@cesar.school)
- JP Veloso (Jpgev@cesar.school)

### **Bibliotecas**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
# Define o dispositivo (GPU se disponível, senão CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### **Define o modelo da rede neural**

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

### **Carrega o dataset Fashion MNIST**

In [ ]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.FashionMNIST('../data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.FashionMNIST('../data', train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size = 64)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size = 1000)


100%|██████████| 26.4M/26.4M [00:02<00:00, 11.4MB/s]


Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 175kB/s]


Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.31MB/s]


Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 22.6MB/s]

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



### **Treinamento e Avaliação do Modelo**

In [ ]:
# Cria uma instância do modelo e move para o dispositivo
model = Net().to(device)

# Define a função de perda e otimizador
# optimizer = optim.Adadelta(model.parameters(), lr=1.0)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treina o modelo
epochs = 3

for epoch in range(1, epochs + 1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# Avalia o modelo
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301761
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.581459
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.388488
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.524741
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.454931
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.516063
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.352203
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.522451
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.406918
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.378471
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.286478
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.375344
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.254475
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.456930
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.337346
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.386357
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.223509
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.517391
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.320124
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.278912
Train Epoch: